In [2]:
from google.colab import files
files.upload()

Saving dev_set_task1.txt to dev_set_task1.txt


{'dev_set_task1.txt': b'[\n    {\n        "id": "62_batch_2",\n        "labels": [],\n        "text": "*President* Biden?\\n\\nPlease, no.\\n"\n    },\n    {\n        "id": "111_batch_2",\n        "labels": [\n            "Loaded Language",\n            "Name calling/Labeling"\n        ],\n        "text": "JOE  VERSUS THE VOLCANIC KREMLIN DON\\n\\n\\"WILL YOU SHUT UP, MAN?\\"\\n\\nLORD OF THE LIES"\n    },\n    {\n        "id": "167_batch_2",\n        "labels": [],\n        "text": "ANTI-VAXXERS BE LIKE... \\n\\nHANG ON A SEC - JUST COLLECTING MY FACTS"\n    },\n    {\n        "id": "93_batch_2",\n        "labels": [],\n        "text": "VIRUS BINGO\\nFREE 32 SPACE\\n"\n    },\n    {\n        "id": "153_batch_2",\n        "labels": [\n            "Exaggeration/Minimisation",\n            "Loaded Language",\n            "Name calling/Labeling"\n        ],\n        "text": "Never thought l\'d die fighting IRRESPONSIBLY RUSHED VACCINES side by side with an ANTIVAXXER\\n\\nWhat about side b

Upload the json file that include the data

In [4]:
import json
f = open('final.json')
data = json.load(f)
data

[{'id': '128',
  'labels': ['Black-and-white Fallacy/Dictatorship'],
  'text': 'THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n'},
 {'id': '128',
  'labels': ['Black-and-white Fallacy/Dictatorship'],
  'text': 'there mare only two genders\n\nfemale \n\nmale\n'},
 {'id': '189', 'labels': [], 'text': 'This is not an accident!'},
 {'id': '96',
  'labels': ['Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears'],
  'text': "SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\n\nPOWER COMES FROM THE BARREL OF A GUN, COMRADES.\n\nWHAT ABOUT THE ONE WHO SHOT CONGRESSMAN SCALISE OR THE DAYTON OHIO MASS SHOOTER?\n"},
 {'id': '96',
  'labels': ['Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears'],
  'text': "so bernie bros haven't committed violence eh?\n\npower comes from the barrel of a gun, comrades.\n\nwhat about the one who shot congressman scalise or the dayton ohio mass shooter?\n"},
 {'id': '154',
  'labels': ['Causal Oversimplification',
   'Loaded Language',
   'Name cal

Find the number of type of labels inside the data

In [5]:
label_type = []
for i in data:
  for label in i['labels']:
    if label not in label_type:
      label_type.append(label)
label_type

['Black-and-white Fallacy/Dictatorship',
 'Loaded Language',
 'Name calling/Labeling',
 'Slogans',
 'Smears',
 'Causal Oversimplification',
 'Appeal to fear/prejudice',
 'Exaggeration/Minimisation',
 'Reductio ad hitlerum',
 'Repetition',
 'Glittering generalities (Virtue)',
 "Misrepresentation of Someone's Position (Straw Man)",
 'Doubt',
 'Obfuscation, Intentional vagueness, Confusion',
 'Whataboutism',
 'Flag-waving',
 'Thought-terminating cliché',
 'Presenting Irrelevant Data (Red Herring)',
 'Appeal to authority',
 'Bandwagon']

Find the maximum of labels that a sentence can include

In [6]:
max = 0
for i in data:
  if len(i['labels'])>max:
    max=len(i['labels'])
print(max)

8


Put the data inside X_train and Y train

In [7]:
X_train = []
Y_train = []
for i in data:
  if len(i['labels'])==0:
    continue
  X_train.append(i['text'])
  train_labels=[]
  for label in i['labels']:
    train_labels.append(label_type.index(label))
  Y_train.append(train_labels)

Get the labels by using multilabel_binarizer

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(Y_train)
labels = multilabel_binarizer.classes_
labels


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [9]:
maxlen = 180
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X_train)
def get_features(text_series):
    """
    transforms text data to feature_vectors that can be used in the ml model.
    tokenizer must be available.
    """
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=maxlen)


def prediction_to_label(prediction):
    tag_prob = [(labels[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=True))


Get the train and test data

In [11]:

x_train = get_features(X_train)
y_train = multilabel_binarizer.transform(Y_train)
print(x.shape)


(1527, 180)


In [12]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

Intialize the model and train the model using kera package

In [134]:
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras import Sequential

filter_length = 300

model = Sequential()
model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=4), 
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=16,
                    validation_split=0.1,
                    callbacks=callbacks)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 180, 20)           100000    
_________________________________________________________________
dropout_18 (Dropout)         (None, 180, 20)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 178, 300)          18300     
_________________________________________________________________
global_max_pooling1d_18 (Glo (None, 300)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 20)                6020      
_________________________________________________________________
activation_18 (Activation)   (None, 20)                0         
Total params: 124,320
Trainable params: 124,320
Non-trainable params: 0
_______________________________________________

In [135]:
import json
test = open('dev_set_task1.txt')
test = json.load(test)
X_test = []
Y_test = []
for i in test:
  if len(i['labels'])==0:
    continue
  X_test.append(i['text'])
  test_labels=[]
  for label in i['labels']:
    test_labels.append(label_type.index(label))
  Y_test.append(test_labels)

In [136]:
x_test = get_features(X_test)
y_test = multilabel_binarizer.transform(Y_test)

In [137]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 11ms/step - loss: 0.3536 - categorical_accuracy: 0.4583


[0.35363471508026123, 0.4583333432674408]

In [150]:
y_pred = model.predict(x_test)

Change the prediction to be binary format

In [151]:
import numpy as np
for i in range(len(y_pred)):
        max_value=np.max(y_pred[i])
        for j in range(len(y_pred[i])):
             if max_value==y_pred[i][j]:
                 y_pred[i][j]=1
             else:
                 y_pred[i][j]=0

Average micro f1 score is 0.3157,Average macro f1 score is 0.2182

In [157]:
from sklearn import metrics
print(metrics.f1_score(y_test,y_pred,average='micro',zero_division=1))
print(metrics.f1_score(y_test,y_pred,average='macro',zero_division=1))

0.3157894736842105
0.21822727272727277
